In [23]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()
from tensorflow.keras.callbacks import EarlyStopping


In [4]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
del application_df['EIN']
del application_df['NAME']

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17', 'T8', 'T7', 'T10']
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [6]:
vc = application_df['CLASSIFICATION'].value_counts()
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = vc[vc<1883].keys()
#print(classifications_to_replace)
# Replace in dataframe
for cls in classifications_to_replace:
    #print(cls)
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
# Convert categorical data to numeric with `pd.get_dummies`
one_hot = pd.get_dummies(application_df, ["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "INCOME_AMT", "SPECIAL_CONSIDERATIONS"])

In [8]:
# Split our preprocessed data into our features and target arrays
X, y = one_hot.drop('IS_SUCCESSFUL', axis=1), one_hot['IS_SUCCESSFUL']
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(input_dim=40, units=6, activation="tanh"))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=6, activation="tanh"))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=3, activation="tanh"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 6)                 246       
                                                                 
 dense_11 (Dense)            (None, 6)                 42        
                                                                 
 dense_12 (Dense)            (None, 3)                 21        
                                                                 
 dense_13 (Dense)            (None, 1)                 4         
                                                                 
Total params: 313 (1.22 KB)
Trainable params: 313 (1.22 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
nn_model.save("/Users/ahlden/DataAnalytics/deep-learning-challenge/model1.h5")

In [20]:
# Train the model
nn_model = tf.keras.models.load_model("/Users/ahlden/DataAnalytics/deep-learning-challenge/model1.h5")
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 3s 3ms/step - loss: 0.6033 - accuracy: 0.6876
Epoch 2/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5707 - accuracy: 0.7222
Epoch 3/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5668 - accuracy: 0.7227
Epoch 4/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5642 - accuracy: 0.7243
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5627 - accuracy: 0.7270
Epoch 6/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5615 - accuracy: 0.7281
Epoch 7/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5606 - accuracy: 0.7281
Epoch 8/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5600 - accuracy: 0.7287
Epoch 9/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5594 - accuracy: 0.7304
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5585 - accuracy: 0.7284

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5516 - accuracy: 0.7319 - 439ms/epoch - 2ms/step
Loss: 0.5516056418418884, Accuracy: 0.7318950295448303


In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn_model2 = tf.keras.models.Sequential()

# First hidden layer
nn_model2.add(tf.keras.layers.Dense(input_dim=40, units=6, activation="tanh"))

# Second hidden layer
nn_model2.add(tf.keras.layers.Dense(units=12, activation="tanh"))

# Third hidden layer
nn_model2.add(tf.keras.layers.Dense(units=6, activation="tanh"))

# Output layer
nn_model2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model2.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 6)                 246       
                                                                 
 dense_23 (Dense)            (None, 12)                84        
                                                                 
 dense_24 (Dense)            (None, 6)                 78        
                                                                 
 dense_25 (Dense)            (None, 1)                 7         
                                                                 
Total params: 415 (1.62 KB)
Trainable params: 415 (1.62 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
# Compile the model
nn_model2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
nn_model2.save("/Users/ahlden/DataAnalytics/deep-learning-challenge/model2.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [32]:
history = nn_model2.fit(X_train_scaled, y_train, epochs=100, batch_size=64,
                        validation_split=0.2, callbacks=[early_stopping], verbose=2)

Epoch 1/100
322/322 - 2s - loss: 0.6246 - accuracy: 0.6655 - val_loss: 0.5701 - val_accuracy: 0.7291 - 2s/epoch - 5ms/step
Epoch 2/100
322/322 - 1s - loss: 0.5808 - accuracy: 0.7121 - val_loss: 0.5602 - val_accuracy: 0.7341 - 614ms/epoch - 2ms/step
Epoch 3/100
322/322 - 1s - loss: 0.5738 - accuracy: 0.7189 - val_loss: 0.5546 - val_accuracy: 0.7378 - 672ms/epoch - 2ms/step
Epoch 4/100
322/322 - 1s - loss: 0.5697 - accuracy: 0.7229 - val_loss: 0.5515 - val_accuracy: 0.7394 - 619ms/epoch - 2ms/step
Epoch 5/100
322/322 - 1s - loss: 0.5677 - accuracy: 0.7237 - val_loss: 0.5510 - val_accuracy: 0.7386 - 660ms/epoch - 2ms/step
Epoch 6/100
322/322 - 1s - loss: 0.5662 - accuracy: 0.7237 - val_loss: 0.5489 - val_accuracy: 0.7415 - 624ms/epoch - 2ms/step
Epoch 7/100
322/322 - 1s - loss: 0.5648 - accuracy: 0.7258 - val_loss: 0.5478 - val_accuracy: 0.7407 - 666ms/epoch - 2ms/step
Epoch 8/100
322/322 - 1s - loss: 0.5634 - accuracy: 0.7240 - val_loss: 0.5475 - val_accuracy: 0.7419 - 678ms/epoch - 2ms/

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5518 - accuracy: 0.7341 - 338ms/epoch - 1ms/step
Loss: 0.5517629981040955, Accuracy: 0.7341107726097107


In [43]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn_model3 = tf.keras.models.Sequential()

# First hidden layer
nn_model3.add(tf.keras.layers.Dense(input_dim=40, units=64, activation="tanh"))

# Second hidden layer
nn_model3.add(tf.keras.layers.Dense(units=32, activation="tanh"))

# Third hidden layer
nn_model3.add(tf.keras.layers.Dense(units=16, activation="tanh"))

# Third hidden layer
nn_model3.add(tf.keras.layers.Dense(units=8, activation="tanh"))

# Third hidden layer
nn_model3.add(tf.keras.layers.Dense(units=4, activation="tanh"))

# Third hidden layer
nn_model3.add(tf.keras.layers.Dense(units=2, activation="tanh"))

# Output layer
nn_model3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model3.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_42 (Dense)            (None, 64)                2624      
                                                                 
 dense_43 (Dense)            (None, 32)                2080      
                                                                 
 dense_44 (Dense)            (None, 16)                528       
                                                                 
 dense_45 (Dense)            (None, 8)                 136       
                                                                 
 dense_46 (Dense)            (None, 4)                 36        
                                                                 
 dense_47 (Dense)            (None, 2)                 10        
                                                                 
 dense_48 (Dense)            (None, 1)                

In [44]:
# Compile the model
nn_model3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
nn_model3.save("/Users/ahlden/DataAnalytics/deep-learning-challenge/model3.h5")

In [45]:
history = nn_model3.fit(X_train_scaled, y_train, epochs=100, batch_size=64,
                        validation_split=0.2, callbacks=[early_stopping], verbose=2)

Epoch 1/100
322/322 - 2s - loss: 0.6000 - accuracy: 0.7103 - val_loss: 0.5668 - val_accuracy: 0.7399 - 2s/epoch - 7ms/step
Epoch 2/100
322/322 - 1s - loss: 0.5776 - accuracy: 0.7240 - val_loss: 0.5548 - val_accuracy: 0.7452 - 745ms/epoch - 2ms/step
Epoch 3/100
322/322 - 1s - loss: 0.5700 - accuracy: 0.7263 - val_loss: 0.5505 - val_accuracy: 0.7429 - 770ms/epoch - 2ms/step
Epoch 4/100
322/322 - 1s - loss: 0.5652 - accuracy: 0.7284 - val_loss: 0.5509 - val_accuracy: 0.7432 - 755ms/epoch - 2ms/step
Epoch 5/100
322/322 - 1s - loss: 0.5633 - accuracy: 0.7271 - val_loss: 0.5454 - val_accuracy: 0.7431 - 760ms/epoch - 2ms/step
Epoch 6/100
322/322 - 1s - loss: 0.5611 - accuracy: 0.7272 - val_loss: 0.5464 - val_accuracy: 0.7423 - 728ms/epoch - 2ms/step
Epoch 7/100
322/322 - 1s - loss: 0.5598 - accuracy: 0.7280 - val_loss: 0.5469 - val_accuracy: 0.7419 - 724ms/epoch - 2ms/step
Epoch 8/100
322/322 - 1s - loss: 0.5586 - accuracy: 0.7292 - val_loss: 0.5456 - val_accuracy: 0.7429 - 704ms/epoch - 2ms/

In [46]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5528 - accuracy: 0.7322 - 344ms/epoch - 1ms/step
Loss: 0.5528221130371094, Accuracy: 0.7322449088096619
